In [1]:
import os
from pathlib import Path
import sys

In [2]:
project_name = 'clpsych'
project_path = Path(os.getcwd()).parent

if sys.platform == "win32":
    data_path = 'D:\Dataset\{0}\dataset'.format(project_name)
    model_path = 'D:\Dataset\{0}\models'.format(project_name)
    src_path = '/Volumes/Dataset/{0}/src'.format(project_name)
    
elif sys.platform == 'darwin':
    data_path = '/Volumes/Dataset/{0}/dataset'.format(project_name)
    model_path = '/Volumes/Dataset/{0}/models'.format(project_name)
    src_path = '/Volumes/Dataset/{0}/src'.format(project_name)
    
else:
    data_path = Path(project_path, 'dataset')
    model_path = Path(project_path, 'models')
    src_path = Path(project_path, 'src')

utils_path = str(Path(project_path, 'utils'))
# including the project folder and the utils folder
if utils_path not in ''.join(sys.path):
    sys.path.extend([str(project_path), utils_path, str(src_path)])

print('project path = {0}'.format(project_path))
print('data path = {0}'.format(data_path))
print('model path = {0}'.format(model_path))
print('sys.path = {0}'.format(sys.path))

project path = /home/guerramarj/github/clpsych
data path = /home/guerramarj/github/clpsych/dataset
model path = /home/guerramarj/github/clpsych/models
sys.path = ['/cm/local/apps/cuda/libs/current/pynvml', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python36.zip', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6/lib-dynload', '', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6/site-packages', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6/site-packages/Mako-1.0.7-py3.6.egg', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6/site-packages/IPython/extensions', '/home/guerramarj/.ipython', '/home/guerramarj/github/clpsych', '/home/guerramarj/github/clpsych/utils', '/home/guerramarj/github/clpsych/src']


In [3]:
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from utils.datapath import data_path_scripts
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import precision_recall_fscore_support
from numpy import interp 
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
import re
import itertools
from collections import Counter
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Dropout
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adamax, Nadam, Adam
from keras.regularizers import l1_l2
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K
from keras import metrics
import tensorflow as tf
import _pickle as pickle 
import stanfordnlp
from keras.layers import LSTM
from keras.preprocessing.text import one_hot
from keras.datasets import imdb 
from utils.data_helpers import load_data_and_labels
from keras.preprocessing.text import text_to_word_sequence
from keras.utils import multi_gpu_model
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

%matplotlib inline
plt.rcParams['figure.figsize'] = [20, 13]

# seed for numpy and sklearn
random_state = 7
np.random.seed(random_state)

Using TensorFlow backend.


In [4]:
# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())
assert 'GPU' in str(device_lib.list_local_devices())
print('Tensorflow recognizes GPUs')

# confirm Keras sees the GPU
from keras import backend
available_gpu = backend.tensorflow_backend._get_available_gpus()
assert len(available_gpu) > 0
available_gpus = len(available_gpu)
print('number of available GPUs = {0}'.format(available_gpus))
print('list of GPUs = {0}\n'.format(available_gpu))

Tensorflow recognizes GPUs
number of available GPUs = 4
list of GPUs = ['/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3']



In [28]:
def class_report(y_true, y_pred, y_score=None, average='macro'):
    if y_true.shape != y_pred.shape:
        print("Error! y_true {0} is not the same shape as y_pred {1}".format(
              y_true.shape,
              y_pred.shape)
        )
        return

    lb = LabelBinarizer()

    if len(y_true.shape) == 1:
        lb.fit(y_true)

    #Value counts of predictions
    labels, cnt = np.unique(
        y_pred,
        return_counts=True)
    n_classes = len(labels)
    pred_cnt = pd.Series(cnt, index=labels)

    metrics_summary = precision_recall_fscore_support(
            y_true=y_true,
            y_pred=y_pred,
            labels=labels)

    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average=average))

    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support']
    class_report_df = pd.DataFrame(
        list(metrics_summary),
        index=metrics_sum_index,
        columns=labels)

    support = class_report_df.loc['support']
    total = support.sum() 
    class_report_df['avg / total'] = avg[:-1] + [total]

    class_report_df = class_report_df.T
    class_report_df['pred-cnt'] = pred_cnt
    class_report_df['pred-cnt'].iloc[-1] = total

    if not (y_score is None):
        # false positive rate
        fpr = dict()
        # true positive rate
        tpr = dict()
        roc_auc = dict()
        for label_ix, label in enumerate(labels):
            fpr[label], tpr[label], _ = roc_curve(
                (y_true == label).astype(int), 
                y_score[:, label_ix])

            roc_auc[label] = auc(fpr[label], tpr[label])

        if average == 'micro':
            if n_classes <= 2:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                    lb.transform(y_true).ravel(), 
                    y_score[:, 1].ravel())
            else:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                        lb.transform(y_true).ravel(), 
                        y_score.ravel())

            roc_auc["avg / total"] = auc(
                fpr["avg / total"], 
                tpr["avg / total"])

        elif average == 'macro':
            # First aggregate all false positive rates
            all_fpr = np.unique(np.concatenate([
                fpr[i] for i in labels]
            ))

            # Then interpolate all ROC curves at this points
            mean_tpr = np.zeros_like(all_fpr)
            for i in labels:
                mean_tpr += interp(all_fpr, fpr[i], tpr[i])

            # Finally average it and compute AUC
            mean_tpr /= n_classes

            fpr["macro"] = all_fpr
            tpr["macro"] = mean_tpr

            roc_auc["avg / total"] = auc(fpr["macro"], tpr["macro"])

        class_report_df['AUC'] = pd.Series(roc_auc)

    return class_report_df

In [6]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [7]:
def auc_metric(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    backend.get_session().run(tf.local_variables_initializer())
    return auc

In [8]:
%ls ../dataset

clpsych19_public_crowd_train.csv         risk_title_body.csv
clpsych19_public_shared_task_posts.csv   sentiment_per_post.csv
clpsych19_public_task_B_train_posts.csv  sentiment_per_user.csv
database.sqlite                          sentiment_per_user_macro.csv
dataset_sentiments.csv                   sentiment_per_user_micro.csv
post_risklabel.csv*                      sentiment_per_user_micro_macro.csv
post_sent_user_risk.csv                  sentiment_user.csv
post_user_risk                           sent_user.csv
post_user_risk.csv                       Tweets.csv
preprocessed_corpus.pkl                  twitter-airline-sentiment.zip
risk_tbs.csv*                            users.csv
risk_tbs_num.csv*


In [21]:
data = pd.read_csv(Path(data_path, 'sentiment_per_user_macro.csv'))
data.head()

,user_id,sent_1_macro,sent_2_macro,sent_3_macro,sent_4_macro,sent_5_macro,risk_label
0,40130,0.169393,0.406493,0.283100,0.103859,0.037155,a
1,19368,0.063450,0.282020,0.427240,0.189121,0.038170,a
2,20841,0.084885,0.370965,0.426645,0.097172,0.020333,a
3,8720,0.132132,0.447101,0.281858,0.108090,0.030819,a
4,32730,0.113888,0.424265,0.250576,0.170719,0.040552,a


In [22]:
data.dropna(subset=['risk_label'], inplace=True)
data.head()

,user_id,sent_1_macro,sent_2_macro,sent_3_macro,sent_4_macro,sent_5_macro,risk_label
0,40130,0.169393,0.406493,0.283100,0.103859,0.037155,a
1,19368,0.063450,0.282020,0.427240,0.189121,0.038170,a
2,20841,0.084885,0.370965,0.426645,0.097172,0.020333,a
3,8720,0.132132,0.447101,0.281858,0.108090,0.030819,a
4,32730,0.113888,0.424265,0.250576,0.170719,0.040552,a


In [23]:
lb_make = LabelEncoder()

y = data[['risk_label']].copy()
y.loc[:, 'risk_label'] = lb_make.fit_transform(y['risk_label'])
y_temp = pd.get_dummies(data['risk_label'].astype('category')).copy()
y[y_temp.keys()] = y_temp
x = data.drop('risk_label', axis=1).copy()

In [41]:
cvscores = list()
auc_scores = list()
report_with_auc_df = ''

# define 10-fold cross validation test harness
kfold = KFold(n_splits=5, shuffle=True, random_state=7)

print('5 fold CV starting')
for cv_ix, x_y in enumerate(kfold.split(x, y)):
    
    print('\nFold = {0}\n'.format(cv_ix))
    
    X_train = x.loc[x_y[0]]
    y_train = y.loc[x_y[0]]
    
    X_test = x.loc[x_y[1]]
    y_test = y.loc[x_y[1]]

#     x_train = X_train.drop(['post_id', 'user_id'], axis=1).copy()
#     x_test = X_test.drop(['post_id', 'user_id'], axis=1).copy()
    x_train = X_train.drop(['user_id'], axis=1).copy()
    x_test = X_test.drop(['user_id'], axis=1).copy()
    y_train_risk = y_train['risk_label'].copy()
    y_test_risk = y_test['risk_label'].copy()
    y_train.drop('risk_label', axis=1, inplace=True)
    y_test.drop('risk_label', axis=1, inplace=True)

    # basic neural network
    model = Sequential()
    model.add(Dense(25, input_dim=np.shape(x_train)[1], activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='sigmoid'))

    model = multi_gpu_model(model, gpus=available_gpus)

    # Compile model
    model.compile(optimizer='adam', loss=f1_loss, metrics=['accuracy', f1, auc_metric])
    model.fit(x_train, y_train, epochs=1, batch_size=100, verbose=1)

    y_pred_train = np.argmax(model.predict(x_train), axis=1)
    y_pred_test = np.argmax(model.predict(x_test), axis=1)

    report_with_auc = class_report(
        y_true=y_test_risk, 
        y_pred=y_pred_test, 
        y_score=model.predict(x_test),
        average='macro')
    
    cv_column = [cv_ix]
    cv_column.extend( [''] * (report_with_auc.index.shape[0] - 1))
    report_with_auc['Fold'] = cv_column
    report_with_auc['Risk-Factor'] = report_with_auc.index
    report_with_auc = report_with_auc.set_index(['Fold', 'Risk-Factor'])

    if cv_ix == 0:
        report_with_auc_df = report_with_auc.copy()
    else:
        report_with_auc_df = report_with_auc_df.append(report_with_auc.copy())

5 fold CV starting

Fold = 0

Epoch 1/1
396/396 [==============================] - 1s 3ms/step - loss: 0.6817 - acc: 0.2121 - f1: 0.2839 - auc_metric: 0.4491

Fold = 1

Epoch 1/1
397/397 [==============================] - 1s 3ms/step - loss: 0.6764 - acc: 0.4257 - f1: 0.3437 - auc_metric: 0.6748

Fold = 2

Epoch 1/1
397/397 [==============================] - 1s 3ms/step - loss: 0.6843 - acc: 0.2267 - f1: 0.1936 - auc_metric: 0.4231

Fold = 3

Epoch 1/1
397/397 [==============================] - 1s 3ms/step - loss: 0.6871 - acc: 0.1310 - f1: 0.0223 - auc_metric: 0.3393

Fold = 4

Epoch 1/1
397/397 [==============================] - 1s 3ms/step - loss: 0.6821 - acc: 0.2594 - f1: 0.1294 - auc_metric: 0.5084


In [42]:
report_with_auc_df

precision    recall  f1-score  support  pred-cnt       AUC
Fold Risk-Factor                                                            
0    1             0.153846  0.333333  0.210526      6.0      13.0  0.556738
     3             0.459770  0.851064  0.597015     47.0      87.0  0.493376
     avg / total   0.153404  0.296099  0.201885     53.0      53.0  0.539043
1    3             0.373737  1.000000  0.544118     37.0      99.0  0.372276
     avg / total   0.093434  0.250000  0.136029     37.0      37.0  0.396469
2    2             0.232323  1.000000  0.377049     23.0      99.0  0.461098
     avg / total   0.058081  0.250000  0.094262     23.0      23.0  0.484840
3    0             0.183673  0.360000  0.243243     25.0      49.0  0.419459
     1             0.128205  0.555556  0.208333      9.0      39.0  0.590123
     2             0.272727  0.136364  0.181818     22.0      11.0  0.469894
     avg / total   0.146151  0.262980  0.158349     56.0      56.0  0.505327
4    0             0.255102  1.000000  0.406504     25.0      98.0  0.471351
     2             1.000000  0.040000  0.076923     25.0       1.0  0.550811
     avg / total   0.313776  0.260000  0.120857     50.0      50.0  0.524324